In [333]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch
import os
import re
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
import spacy
from nltk.stem import WordNetLemmatizer
import deepl

In [98]:
# Load the files 
question_globales_path = 'data/QuestionGlobales.xlsx'
extraction_codebook_path = 'data/Extraction CodeBook - 3. Cleaned.xlsx'

# Read the global questions file
df_globales = pd.read_excel(question_globales_path)

# Read all sheets from the codebook file
sheets_codebook = pd.read_excel(extraction_codebook_path, sheet_name=None)



In [99]:
sheets_codebook.keys()

dict_keys(['1988', '1994', '1998', '2005', '2009', '2017', '2023'])

In [142]:
# Créer des DataFrames pour chaque année
years = ['1988', '1994', '1998', '2005', '2009', '2017', '2023']
dataframes = {}
columns_to_keep = ['code', 'num_question', 'year', 'label', 'type', 'format']


for year in years:
    if year in sheets_codebook:  # Vérifier si la feuille existe
        # Chaque feuille est un DataFrame
        df = sheets_codebook[year]
        # Garder uniquement les colonnes spécifiées
        dataframes[f'main_questions_{year}'] = df[columns_to_keep]
    else:
        print(f"Feuille pour l'année {year} non trouvée.")

# Accéder aux DataFrames pour chaque année
main_questions_1988 = dataframes.get('main_questions_1988', pd.DataFrame())
main_questions_1994 = dataframes.get('main_questions_1994', pd.DataFrame())
main_questions_1998 = dataframes.get('main_questions_1998', pd.DataFrame())
main_questions_2005 = dataframes.get('main_questions_2005', pd.DataFrame())
main_questions_2009 = dataframes.get('main_questions_2009', pd.DataFrame())
main_questions_2017 = dataframes.get('main_questions_2017', pd.DataFrame())
main_questions_2023 = dataframes.get('main_questions_2023', pd.DataFrame())

main_questions_2009.head()

,code,num_question,year,label,type,format
0,GSB09_1,1,2009,"Es kann Probleme geben, welche die Gemei...",NaN,NaN
1,GSB09_1_1,1,2009,Unterstützung und Betreuung älterer Menschen,NaN,NaN
2,GSB09_1_2,1,2009,Jugendfragen,NaN,NaN
3,GSB09_1_3,1,2009,familienergänzende Kinderbetreuung,NaN,NaN
4,GSB09_1_4,1,2009,Unterstützung und Betreuung von Arbeitslosen,NaN,NaN


In [155]:
def filter_num_question_old(df):
    # Supprimer les lignes avec des NaN dans 'num_question'
    df = df[df['num_question'].notna()].copy()  # Créer une copie pour éviter l'avertissement

    # Convertir 'num_question' en chaîne
    df.loc[:, 'num_question'] = df['num_question'].astype(str)  # Utiliser .loc pour éviter l'avertissement

    # Filtrer les lignes où num_question est composé uniquement de chiffres ou contient 'a'/'A'
    df = df[df['num_question'].str.match(r'^\d+$|.*[aA].*')]
    
    # Garder seulement la première occurrence pour chaque num_question
    df = df.drop_duplicates(subset='num_question', keep='first')

    df['num_question'] = df['num_question'].str.replace('a', '', regex=False).str.replace('A', '', regex=False)

    df = df.drop_duplicates(subset='num_question', keep='first')

    return df

In [141]:
def filter_num_question_new(df):
    # Supprimer les lignes avec des NaN dans 'num_question'
    df = df[df['num_question'].notna()].copy()  # Créer une copie pour éviter l'avertissement
    df = df.drop_duplicates(subset='num_question', keep='first')
    return df

In [156]:
main_questions_1988 = filter_num_question_old(main_questions_1988)
main_questions_1994 = filter_num_question_old(main_questions_1994)
main_questions_1998 = filter_num_question_old(main_questions_1998)
main_questions_2005 = filter_num_question_old(main_questions_2005)
main_questions_2009 = filter_num_question_old(main_questions_2009)
main_questions_2017 = filter_num_question_new(main_questions_2017)
main_questions_2017 = main_questions_2017[main_questions_2017['num_question'] != 'Q1']
main_questions_2023 = filter_num_question_new(main_questions_2023)


In [160]:
main_questions_1988.head()

,code,num_question,year,label,type,format
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN
5,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN
7,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN
8,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN
20,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN


In [158]:
dataframes_to_merge = [
    main_questions_1988,
    main_questions_1994,
    main_questions_1998,
    main_questions_2005,
    main_questions_2009,
    main_questions_2017,
    main_questions_2023
]

# Fusionner tous les DataFrames
merged_df = pd.concat(dataframes_to_merge, ignore_index=True)

In [159]:
merged_df.head()

,code,num_question,year,label,type,format
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN


text filtering

In [230]:
!python -m spacy download de_core_news_sm

     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
     --------------------------------------- 0.0/14.6 MB 320.0 kB/s eta 0:00:46
     ---------------------------------------- 0.1/14.6 MB 1.0 MB/s eta 0:00:15
      --------------------------------------- 0.2/14.6 MB 1.8 MB/s eta 0:00:08
     - -------------------------------------- 0.4/14.6 MB 2.2 MB/s eta 0:00:07
     - -------------------------------------- 0.6/14.6 MB 2.6 MB/s eta 0:00:06
     -- ------------------------------------- 0.8/14.6 MB 3.0 MB/s eta 0:00:05
     -- ------------------------------------- 1.0/14.6 MB 3.1 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/14.6 MB 3.3 MB/s eta 0:00:05
     --- ------------------------------------ 1.4/14.6 MB 3.5 MB/s eta 0:00:04
     ---- ----------------------------------- 1.7/14.6 MB 3.7 MB/s eta 0:00:04
     ----- ---------------------------------- 1.9/14.6 MB 3.7 MB/s eta 0:00:04
     ----- ---------------------------------- 2.1/14.6 MB 

In [233]:
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # 1. Conversion en minuscules
    text = text.lower()
    
    # 2. Suppression de la ponctuation
    text = re.sub(r'[^\w\s]', '', text)

    # 3. Suppression des espaces supplémentaires
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 4. Traitement avec spaCy pour lemmatisation, stopword removal et tokenization
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]  # Lemmatisation et suppression des stopwords
    
    return tokens  # Retourner la liste de tokens

# Appliquer le prétraitement sur la colonne 'label' de merged_df
tqdm.pandas()  # Activer tqdm pour le suivi de la progression
merged_df['tokens'] = merged_df['label'].progress_apply(preprocess_text)


100%|██████████| 563/563 [00:03<00:00, 185.28it/s]


In [234]:
merged_df.head()

,code,num_question,year,label,type,format,cluster,processed_label,tokens
0,GSB88_1a,1,1988,Postleitzahl der Gemeinde,NaN,NaN,-1,Postleitzahl Gemeinde,"[Postleitzahl, Gemeinde]"
1,GSB88_2a,2,1988,Wieviele Einwohner umfasst ihre Gemeinde,NaN,NaN,-1,wieviel Einwohner umfassen Gemeinde,"[wieviele, Einwohner, umfassen, Gemeinde]"
2,GSB88_3,3,1988,Welche Beschreibung trifft auf Ihre Gemeinde a...,NaN,NaN,-1,Beschreibung treffen Gemeinde --,"[Beschreibung, treffen, Gemeinde]"
3,GSB88_4,4,1988,Welche dieser Bezeichnungen treffen am ehesten...,NaN,NaN,-1,Bezeichnung treffen eher Gemeinde --,"[Bezeichnung, Treffen, eher, Gemeinde]"
4,GSB88_5a,5,1988,Wenn man das Zusammenleben in einer Gemeinde b...,NaN,NaN,-1,zusammenleben Gemeinde betrachten -- Extreme...,"[zusammenleben, Gemeinde, betrachten, extrem, ..."


Now, add nlp pipeline --> bert transformer version 

In [235]:
model_name = 'dbmdz/bert-base-german-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [236]:
def get_embeddings(labels):
    embeddings = []
    for label in tqdm(labels, desc='Generating embeddings'):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Utiliser le vecteur [CLS] comme embedding du label
            embedding = outputs.last_hidden_state[:, 0, :].numpy()
            embeddings.append(embedding)
    return np.vstack(embeddings)


In [257]:
# Obtenir les tokens du DataFrame fusionné
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()  # Convertir les listes de tokens en chaînes

# Obtenir les embeddings des tokens
embeddings = get_embeddings(labels)

# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(embeddings)

# Créer un DataFrame pour les similarités
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# Seuil de similarité pour regrouper les questions similaires (par exemple, 0.99)
threshold = 0.97

similar_pairs = []

# Identifier les paires de tokens similaires avec tqdm pour la barre de progression
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):  # Pas besoin de tqdm ici car l'intérieur est déjà itéré
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# Construire le nouveau DataFrame question_globale_new
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # Ajouter une ligne pour chaque paire de tokens similaires
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  # Ajouter l'identifiant
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }
    # Utiliser pd.concat au lieu de append
    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 2044.76it/s]


In [258]:
question_globale_new.head()

,id,label,year,code
0,1,wieviele administrativ tätig angestellter Beam...,"1988, 1994","GSB88_8, GSB94_39a"
1,2,stark persönlich Einschätzung allgemein einflu...,"1988, 2005, 2009","GSB88_42, GSB05_30, GSB09_29"
2,3,steuerfuss Steueranlage Gemeinde Vergleich 199...,"1994, 1998, 2005","GSB94_15, GSB98_11, GSB05_3"
3,4,steuerfuss Steueranlage Gemeinde Vergleich 199...,"1994, 1998, 2009","GSB94_15, GSB98_11, GSB09_2"
4,5,real Ertrag Einkommen Vermögenssteuer Gemeinde...,"1994, 1998, 2005","GSB94_16, GSB98_12, GSB05_4"


In [259]:
question_globale_new.shape

(43, 4)

In [260]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei, wieviele administrativ tätig angestellter Beamter Lehrlinge Gemeindeverwaltung beschäftigen respektive 10 beschäftigen Personal gemeindeeigen Werk anstalen Bau Strassenarbeiter abwart Polizei'

version avec dbscan

In [277]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances
import pandas as pd
import numpy as np


labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# Fonction pour obtenir les embeddings
def get_embeddings(labels):
    embeddings = []
    # Utiliser tqdm pour afficher la progression
    for label in tqdm(labels, desc="Calcul des embeddings", unit="label"):
        inputs = tokenizer(label, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state[:, 0, :].numpy()  # Utiliser le vecteur [CLS]
            embeddings.append(embedding)
    return np.vstack(embeddings)

# Obtenir les labels uniques
labels = merged_df['label'].unique()

# Obtenir les embeddings des labels
embeddings = get_embeddings(labels)
print("Shape of embeddings:", embeddings.shape)

# Calculer la matrice de distance (utiliser la distance cosinus)
distance_matrix = cosine_distances(embeddings)

# Appliquer DBSCAN sur la matrice de distance
dbscan = DBSCAN(eps=0.05, min_samples=3, metric='precomputed')  # Utilisez 'precomputed' pour la matrice de distance
clusters = dbscan.fit_predict(distance_matrix)

# Créer un dictionnaire pour mapper les labels aux indices dans merged_df
label_to_index = {label: index for index, label in enumerate(merged_df['label'].unique())}
# Créer une série pour stocker les clusters
cluster_series = pd.Series(-1, index=merged_df.index)

# Assigner les clusters en utilisant le mappage
for label, cluster_id in zip(labels, clusters):
    index = label_to_index[label]
    cluster_series.iloc[index] = cluster_id

# Ajouter les clusters au DataFrame d'origine
merged_df['cluster'] = cluster_series

# Créer un nouveau DataFrame pour les questions globales
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

# Remplir le DataFrame avec les clusters
id_counter = 1
for cluster_id in set(merged_df['cluster']):
    if cluster_id != -1:  # Ignorer le bruit
        temp_df = merged_df[merged_df['cluster'] == cluster_id]
        if not temp_df.empty:
            combined_row = {
                'id': id_counter,
                'label': '; '.join(temp_df['label']),
                'year': '; '.join(temp_df['year'].astype(str)),
                'code': '; '.join(temp_df['code'])
            }
            question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)
            id_counter += 1

# Vérifiez le résultat
question_globale_new.head()

Calcul des embeddings: 100%|██████████| 549/549 [00:25<00:00, 21.64label/s]

Shape of embeddings: (549, 768)


,id,label,year,code
0,1,Wenn in Ihrer Gemeinde das Amt des Schreibers ...,"1988, 1988, 1988, 1988, 1988, 1988, 1988, 1988...","GSB88_9, GSB88_11, GSB88_21a, GSB88_22a, GSB88..."
1,2,Wurde von einer Partei bei den letzten allgeme...,"1988, 1988, 2005, 2009, 2009, 2017","GSB88_27a, GSB88_28a, GSB05_41a1, GSB09_37, GS..."
2,3,Wurden in den letzten 5 Jahren Entscheid...,"1988, 1994, 2005","GSB88_38, GSB94_30a, GSB05_22a"
3,4,Nach welchem Wahlverfahren wird das Parlament ...,"1988, 1998, 2017, 2017, 2017","GSB88_39, GSB98_40, GSB17_Q27, GSB17_Q135_2, G..."
4,5,Wie würden Sie die Zusammenarbeit zw...,"1988, 1988, 1994, 1994, 1994, 1994, 1998, 1998...","GSB88_47, GSB88_57, GSB94_13a16, GSB94_14a, GS..."


In [278]:
question_globale_new.shape

(18, 4)

In [279]:
question_globale_new.head()

,id,label,year,code
0,1,Wenn in Ihrer Gemeinde das Amt des Schreibers ...,"1988, 1988, 1988, 1988, 1988, 1988, 1988, 1988...","GSB88_9, GSB88_11, GSB88_21a, GSB88_22a, GSB88..."
1,2,Wurde von einer Partei bei den letzten allgeme...,"1988, 1988, 2005, 2009, 2009, 2017","GSB88_27a, GSB88_28a, GSB05_41a1, GSB09_37, GS..."
2,3,Wurden in den letzten 5 Jahren Entscheid...,"1988, 1994, 2005","GSB88_38, GSB94_30a, GSB05_22a"
3,4,Nach welchem Wahlverfahren wird das Parlament ...,"1988, 1998, 2017, 2017, 2017","GSB88_39, GSB98_40, GSB17_Q27, GSB17_Q135_2, G..."
4,5,Wie würden Sie die Zusammenarbeit zw...,"1988, 1988, 1994, 1994, 1994, 1994, 1998, 1998...","GSB88_47, GSB88_57, GSB94_13a16, GSB94_14a, GS..."


In [280]:
question_globale_new[question_globale_new['id'] == 1].label.values[0]

'Wenn in Ihrer Gemeinde das Amt des Schreibers (oder eines anderen Chefbeamten)    neu    besetzt    werden    soll:    wird    da    auf    die\nParteimitgliedschaft der Bewerber geachtet?, Ist   der   Präsident   (LU:   Ammann)   Ihrer   Gemeinde   vollamtlich, halbamtlich oder nebenamtlich tätig?, Sind  Selbständigerwerbende  im Gemeindevorstand  (inkl.  Präsident)\nvertreten?, Befinden sich unter den Vorstandsmitgliedern (inkl. Präsident) auch Mandatsträger auf kantonaler oder nationaler Ebene?, Wieviele Kandidaten haben sich insgesamt bei den letzten ordent- lichen  Wahlen  um  Sitze  im  Gemeindevorstand  beworben,  und  von\nwelcher Partei (Partei und Anzahl) wurden sie gestellt?, Wie oft kommt es vor, dass Vorstandsmitglieder, die derselben Partei angehören,    bei    Sachentscheiden    gegensätzliche    Auffassungen\nvertreten?, Gab   es   bei   den   letzten   oder   vorletzten   allgemeinen   Wahlen amtierende Vorstandsmitglieder, die erneut kandidierten, aber nicht\nwiederg

nouvelle version avec tf-idf

In [319]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Obtenir les tokens du DataFrame fusionné et les convertir en chaînes
labels = merged_df['tokens'].apply(lambda x: ' '.join(x)).unique()

# Créer le modèle TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(labels)

# Calculer la similarité cosinus
similarity_matrix = cosine_similarity(tfidf_matrix)

# Créer un DataFrame pour les similarités
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# Seuil de similarité pour regrouper les questions similaires (par exemple, 0.99)
threshold = 0.8

similar_pairs = []

# Identifier les paires de tokens similaires avec tqdm pour la barre de progression
for i in tqdm(range(len(labels)), desc='Identifying similar pairs'):
    for j in range(i + 1, len(labels)):
        if similarity_matrix[i][j] >= threshold:
            similar_pairs.append((labels[i], labels[j]))

# Construire le nouveau DataFrame question_globale_new
question_globale_new = pd.DataFrame(columns=['id', 'label', 'year', 'code'])

id_counter = 1

for label1, label2 in similar_pairs:
    # Ajouter une ligne pour chaque paire de tokens similaires
    temp_df = merged_df[(merged_df['tokens'].apply(lambda x: ' '.join(x)) == label1) | 
                        (merged_df['tokens'].apply(lambda x: ' '.join(x)) == label2)]
    combined_row = {
        'id': id_counter,  # Ajouter l'identifiant
        'label': f"{label1}; {label2}",
        'year': '; '.join(temp_df['year'].astype(str)),
        'code': '; '.join(temp_df['code'])
    }
    # Utiliser pd.concat au lieu de append
    question_globale_new = pd.concat([question_globale_new, pd.DataFrame([combined_row])], ignore_index=True)

    id_counter += 1


Identifying similar pairs: 100%|██████████| 514/514 [00:00<00:00, 15251.80it/s]


In [320]:
question_globale_new.shape

(73, 4)

In [321]:
question_globale_new.head()

,id,label,year,code
0,1,wieviele administrativ tätig angestellter Beam...,1988; 1994,GSB88_8; GSB94_39a
1,2,politisch Auseinandersetzung Gemeinde Auge hal...,1988; 1994; 2005,GSB88_52; GSB94_21; GSB05_29
2,3,Wahlbeteiligung Gemeinde; Wahlbeteiligung Geme...,1988; 1998; 2017,GSB88_55; GSB98_43; GSB17_Q35
3,4,Gemeinde unterschiedlich Masse bestimmt gesell...,1994; 1998; 2005,GSB94_5; GSB98_3; GSB05_1
4,5,Problem Gemeinde aufgrund hoch Arbeitsbelastun...,1994; 1998; 2005; 2009,GSB94_8; GSB98_5; GSB05_2; GSB09_1


In [322]:
question_globale_new[question_globale_new['id'] == 5].label.values[0]

'Problem Gemeinde aufgrund hoch Arbeitsbelastung mangelnd Fachkompetenz angemessen Weise bewältigen Bereich Gemeinde leistungsgrenzen lg sichtbar; Problem geben Gemeinde aufgrund hoch Arbeitsbelastung mangelnd Fachkompetenz angemessen Weise bewältigen Gemeinde leistungsgrenzen lg folgend aufgabenbereich sichtbar'

bien meilleur 

csv extraction of the global questions

In [323]:
quest_glob_columns = [
    'label', 'code_first_question','text_de', 'text_fr', 'text_it', 'text_ro', 'text_en',
    'category_label', 'category_text_de', 'category_text_fr',
    'category_text_it', 'category_text_ro', 'category_text_en',
    'options_value', 'options_label'
]

code_to_token = {}

for index, row in merged_df.iterrows():
    code = row['code']
    tokens = row['tokens']
    if code not in code_to_token and tokens:  # Vérifier que le code n'est pas déjà dans le dictionnaire
        code_to_token[code] = tokens[0]


print(code_to_token)

{'GSB88_1a': 'Postleitzahl', 'GSB88_2a': 'wieviele', 'GSB88_3': 'Beschreibung', 'GSB88_4': 'Bezeichnung', 'GSB88_5a': 'zusammenleben', 'GSB88_6': 'Gemeinde', 'GSB88_7': 'wieviele', 'GSB88_8': 'wieviele', 'GSB88_9': 'Gemeinde', 'GSB88_10a': 'Gemeindeschreiber', 'GSB88_11': 'Präsident', 'GSB88_12a': 'Präsident', 'GSB88_13': 'Partei', 'GSB88_14a': 'Kandidat', 'GSB88_15': 'Ort', 'GSB88_16': 'wahlverfahren', 'GSB88_17a': 'Anzahl', 'GSB88_19a': 'Frau', 'GSB88_20a': 'finden', 'GSB88_21a': 'Selbständigerwerbend', 'GSB88_22a': 'befinden', 'GSB88_23a': 'wieviele', 'GSB88_24': 'möchten', 'GSB88_25': 'vorstandsmitglied', 'GSB88_26a': 'letzter', 'GSB88_27a': 'Partei', 'GSB88_28a': 'Partei', 'GSB88_29': 'letzter', 'GSB88_30': 'Gemeinde', 'GSB88_31a': 'wieviele', 'GSB88_32': 'häufig', 'GSB88_33a': 'bestehen', 'GSB88_34': 'Gemeinde', 'GSB88_35': 'Gemeinde', 'GSB88_36a': 'wievieler', 'GSB88_37': 'haufig', 'GSB88_38': 'letzter', 'GSB88_39': 'wahlverfahren', 'GSB88_40a': 'festgelegt', 'GSB88_41': 'Gemein

In [330]:
# Créer le DataFrame final avec les colonnes spécifiées
df_fin = pd.DataFrame(columns=quest_glob_columns)

# Remplir df_fin avec le premier code et le premier token de chaque question
for index, row in question_globale_new.iterrows():
    code_first_question = row['code'].split('; ')[0]  # Prendre seulement le premier code
    text_de_first_question = row['label'].split('; ')[0]
    first_token = code_to_token.get(code_first_question, '')  # Obtenir le premier token associé

    # Créer une nouvelle ligne
    new_row = {
        'label': first_token,  # Utiliser le premier code dans 'label'
        'code_first_question': code_first_question,  # Mettre le code dans 'code_first_question'
        'text_de': text_de_first_question,  # Utiliser le label de question dans 'text_de'
        'text_fr': '',  # Remplir avec des chaînes vides ou des valeurs par défaut
        'text_it': '',
        'text_ro': '',
        'text_en': '',
        'category_label': '',
        'category_text_de': '',
        'category_text_fr': '',
        'category_text_it': '',
        'category_text_ro': '',
        'category_text_en': '',
        'options_value': '',
        'options_label': ''
    }

    # Ajouter la nouvelle ligne au DataFrame
    df_fin = pd.concat([df_fin, pd.DataFrame([new_row])], ignore_index=True)


df_fin.head()


,label,code_first_question,text_de,text_fr,text_it,text_ro,text_en,category_label,category_text_de,category_text_fr,category_text_it,category_text_ro,category_text_en,options_value,options_label
0,wieviele,GSB88_8,wieviele administrativ tätig angestellter Beam...,,,,,,,,,,,,
1,politisch,GSB88_52,politisch Auseinandersetzung Gemeinde Auge hal...,,,,,,,,,,,,
2,Wahlbeteiligung,GSB88_55,Wahlbeteiligung Gemeinde,,,,,,,,,,,,
3,Gemeinde,GSB94_5,Gemeinde unterschiedlich Masse bestimmt gesell...,,,,,,,,,,,,
4,Problem,GSB94_8,Problem Gemeinde aufgrund hoch Arbeitsbelastun...,,,,,,,,,,,,


In [331]:
df_fin[df_fin['code_first_question'] == 'GSB94_5'].text_de.values

array(['Gemeinde unterschiedlich Masse bestimmt gesellschaftlich Entwicklung betreffen nachfolgend Reihe Entwicklung auflisten bitte geben Ausmass Gemeinde betreffen'],
      dtype=object)